In [1]:
import tensorflow as tf
import numpy as np
import cv2

In [2]:
model = tf.keras.models.load_model(f"./Best1.h5")

In [5]:
c =[[0,0,0],
    [0,0,255],
    [127,127,127],
    [255,255,255],
    [255,0,0]]
def applycolor(img,color):
  #클래스 이미지에 색 적용
  h,w,c = img.shape

  for i in range(h):
    for j in range(w):
      for k in range(c):
        img[i,j,k] = color[img[i,j,k]][k]

  return img

In [6]:

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        original_size = img.shape

        cp_img = cv2.resize(img,(480,320))
        cp_img = cv2.cvtColor(cp_img,cv2.COLOR_RGB2BGR)
        cp_img = cp_img / 255.0
        cp_img = np.expand_dims(cp_img,0)

        p = model.predict(cp_img,verbose=0)
        p = p[0]
        p = np.argmax(p,-1)
        p = np.expand_dims(p,-1)
        p = np.concatenate([p,p,p],-1)
        p = applycolor(p,c)
        p = p.astype(np.float32)
        p = cv2.resize(p,(original_size[1],original_size[0]),interpolation=cv2.INTER_LINEAR)

        result = cv2.addWeighted(img.astype(np.uint8), 1, p.astype(np.uint8), 0.5,0)

        if ret:
            cv2.imshow("camera",result)
            if cv2.waitKey(1) != -1:
                break
        else:
            print("no fram")
            break
else:
    print("can't open camera")
cap.release()
cv2.destroyAllWindows()